In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
from torchinfo import summary
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
pip install torchinfo


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -o -q "/content/drive/My Drive/realzip.zip" -d "/content/dataset"



In [ ]:
normal_folder_path = "/content/dataset/Data/Normal"
tumor_folder_path = "/content/dataset/Data/Tumor"


train_size = 0.80  #80 data 20 valid
batch_size = 32
num_epochs = 10
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((256, 256)),  #resizee
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
file_paths = []
labels = []

for subdir, dirs, files in os.walk(tumor_folder_path):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(subdir, file)
            file_paths.append(file_path)
            if "glioma_tumor" in file_path:
                labels.append('Glioma')
            elif "meningioma_tumor" in file_path:
                labels.append('Meningioma')
            elif "pituitary_tumor" in file_path:
                labels.append('Pituitary')

normal_file_paths = []
for file in os.listdir(normal_folder_path):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        file_path = os.path.join(normal_folder_path, file)
        normal_file_paths.append(file_path)
        labels.append('Normal')
#label and sorting

file_paths.extend(normal_file_paths)

df_tumors = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})
df_tumors.shape

(21672, 2)

In [ ]:
df_tumors.head(5)

,file_path,label
0,/content/dataset/Data/Tumor/pituitary_tumor/P_...,Pituitary
1,/content/dataset/Data/Tumor/pituitary_tumor/P_...,Pituitary
2,/content/dataset/Data/Tumor/pituitary_tumor/P_...,Pituitary
3,/content/dataset/Data/Tumor/pituitary_tumor/P_...,Pituitary
4,/content/dataset/Data/Tumor/pituitary_tumor/P_...,Pituitary


In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        label_mapping = {'Normal': 0, 'Glioma': 1, 'Meningioma': 2, 'Pituitary': 3}
        label = torch.tensor(label_mapping[label], dtype=torch.long)

        return image, label


In [ ]:
train_df, val_df = train_test_split(df_tumors, test_size=(1-train_size), random_state=42, stratify=df_tumors['label'])


In [ ]:

train_dataset = BrainTumorDataset(dataframe=train_df, transform=transform)
val_dataset = BrainTumorDataset(dataframe=val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
num_train_images = len(train_loader.dataset)
num_val_images = len(val_loader.dataset)

print(f"Number of images in the training dataloader: {num_train_images}")
print(f"Number of images in the validation dataloader: {num_val_images}")

Number of images in the training dataloader: 17337
Number of images in the validation dataloader: 4335


In [ ]:
def resNet50():
    ResNet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    for param in ResNet50.parameters():
        param.requires_grad = False

    for param in ResNet50.fc.parameters():
        param.requires_grad = True

    in_features = ResNet50.fc.in_features
    ResNet50.fc = nn.Linear(in_features, 4)

    return ResNet50.to(device)

In [ ]:
model = resNet50()
#summary not needed
summary(model, input_size=(32, 3, 256, 256), col_names=["input_size", "output_size", "num_params", "trainable(Bool)"], col_width=20, row_settings=["var_names"])

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 174MB/s]


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 256, 256]    [32, 4]              --                   Partial
├─Conv2d (conv1)                         [32, 3, 256, 256]    [32, 64, 128, 128]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 128, 128]   [32, 64, 128, 128]   (128)                False
├─ReLU (relu)                            [32, 64, 128, 128]   [32, 64, 128, 128]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 128, 128]   [32, 64, 64, 64]     --                   --
├─Sequential (layer1)                    [32, 64, 64, 64]     [32, 256, 64, 64]    --                   False
│    └─Bottleneck (0)                    [32, 64, 64, 64]     [32, 256, 64, 64]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 64, 64]     [32, 64, 64, 64]     (4,096)              False
│    │    

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

best_accuracy = 0.0
best_model_path = "/content/best_model.pth"
train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

for epoch in range(num_epochs):
    model.train()
    total_loss, correct_train, total_train = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)


        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()


    train_loss = total_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)


    model.eval()
    val_loss, correct_val, total_val = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved better model with accuracy: {val_accuracy:.2f}%")

Epoch [1/10], Train Loss: 0.7232, Validation Loss: 0.5632, Train Accuracy: 71.89%, Validation Accuracy: 78.09%
Saved better model with accuracy: 78.09%
Epoch [2/10], Train Loss: 0.5594, Validation Loss: 0.5043, Train Accuracy: 78.92%, Validation Accuracy: 82.28%
Saved better model with accuracy: 82.28%
Epoch [3/10], Train Loss: 0.5225, Validation Loss: 0.4967, Train Accuracy: 80.03%, Validation Accuracy: 82.56%
Saved better model with accuracy: 82.56%
Epoch [4/10], Train Loss: 0.5038, Validation Loss: 0.4676, Train Accuracy: 80.79%, Validation Accuracy: 82.45%
Epoch [5/10], Train Loss: 0.4757, Validation Loss: 0.4540, Train Accuracy: 81.77%, Validation Accuracy: 83.90%
Saved better model with accuracy: 83.90%
Epoch [6/10], Train Loss: 0.4715, Validation Loss: 0.4186, Train Accuracy: 81.96%, Validation Accuracy: 84.73%
Saved better model with accuracy: 84.73%
Epoch [7/10], Train Loss: 0.4651, Validation Loss: 0.3952, Train Accuracy: 82.05%, Validation Accuracy: 86.04%
Saved better model

In [ ]:
from PIL import Image
from torchvision import transforms
import torch
import torch.nn.functional as F

def predict_image(image_path, model_path):
    model = resNet50()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)


    model.eval()

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    image = image.to(device)



    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)


#? i thinK?


        _, predicted = torch.max(probabilities, 1)
        confidence = probabilities[0][predicted.item()].item()

    classes = ['Normal', 'Glioma', 'Meningioma', 'Pituitary']
    result = classes[predicted[0]]
    print(f"The image is classified as: {result} with a confidence of: {confidence:.2f}")


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving M_100_RO_.jpg to M_100_RO_.jpg


In [ ]:

best_model_path = "/content/best_model.pth"

image_path = "/content/M_100_RO_.jpg"


predict_image(image_path, best_model_path)

The image is classified as: Meningioma with a confidence of: 0.87


In [ ]:
torch.save(model, '/content/brain_tumor_model_include_four_classification_confidence.pth')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
torch.save(model, '/content/drive/My Drive/brain_tumor_model_include_four_classification_confidence.pth')
